In [1]:
import os 
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Conv2D, MaxPool2D,Flatten,Dropout 
from keras.callbacks import EarlyStopping 
from tqdm import tqdm
import keras

In [2]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224


In [3]:
DATADIR = 'D1/train'
CATEGORIES = ["qcerris", "qfrainetto"]
x_train = []
y_train = []
for category in CATEGORIES:  
    path = os.path.join(DATADIR,category) 
    class_num = CATEGORIES.index(category) 
    for img in tqdm(os.listdir(path)): 
        img_array = cv2.imread(os.path.join(path,img)) 
        new_array = cv2.resize(img_array, (IMAGE_WIDTH, IMAGE_HEIGHT)) 
        x_train.append(new_array)
        y_train.append(class_num)  


100%|██████████| 52/52 [00:00<00:00, 84.72it/s]


In [4]:
DATADIR = 'D1/test'
x_test = []
y_test = []
for category in CATEGORIES:  
    path = os.path.join(DATADIR,category) 
    class_num = CATEGORIES.index(category) 
    for img in tqdm(os.listdir(path)): 
        img_array = cv2.imread(os.path.join(path,img)) 
        new_array = cv2.resize(img_array, (IMAGE_WIDTH, IMAGE_HEIGHT)) 
        x_test.append(new_array)
        y_test.append(class_num)  


100%|██████████| 12/12 [00:00<00:00, 76.48it/s]


In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range = 40,
        shear_range = 0.3,
        zoom_range = 0.3,
        horizontal_flip = True,
        brightness_range = (0.5, 1.5))

x_train = np.array(x_train)
y_train = np.array(y_train)
augmented_images = []
augmented_labels = []
    
for i in range(len(x_train)):
        img = x_train[i].reshape((1,) + x_train[i].shape)
        for batch in datagen.flow(img, batch_size=1):
            augmented_images.append(batch[0])
            augmented_labels.append(y_train[i])
            if len(augmented_images) >= 2:  # Adjust the number of augmentations per image
                break

x_train = np.concatenate([x_train, np.array(augmented_images)])
y_train = np.concatenate([y_train, np.array(augmented_labels)])



In [6]:
x_test = np.array(x_test)
y_test = np.array(y_test)
augmented_images = []
augmented_labels = []
    
for i in range(len(x_test)):
        img = x_test[i].reshape((1,) + x_test[i].shape)
        for batch in datagen.flow(img, batch_size=1):
            augmented_images.append(batch[0])
            augmented_labels.append(y_test[i])
            if len(augmented_images) >= 2:  # Adjust the number of augmentations per image
                break

x_test = np.concatenate([x_test, np.array(augmented_images)])
y_test = np.concatenate([y_test, np.array(augmented_labels)])


print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (209, 224, 224, 3)
y_train shape: (209,)
x_test shape: (49, 224, 224, 3)
y_test shape: (49,)


In [7]:
import random 
# Shuffle the dataset
num_samples = len(x_train)
indices = np.random.permutation(num_samples)
x_train = x_train[indices]
y_train = y_train[indices]
# Shuffle the dataset
num_samples = len(x_test)
indices = np.random.permutation(num_samples)
x_test = x_test[indices]
y_test = y_test[indices]

In [8]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [9]:
import numpy as np

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.astype(int)].T
    return Y.T

# One-hot encode the target labels
y_train = convert_to_one_hot(y_train, 2)

# One-hot encode the target labels
y_test = convert_to_one_hot(y_test, 2)



In [10]:
x_train = x_train / 255.0

x_test = x_test / 255.0



In [11]:
from keras.applications import VGG16
input_shape = (224, 224, 3)

model = Sequential()
VGG16_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

for layer in VGG16_model.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False  # Not trainable weights

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))  
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping_monitor = EarlyStopping(patience=3, verbose = 1)
model.fit(x_train, y_train, epochs=10, validation_split = 0.3, callbacks=[early_stopping_monitor])


Epoch 1/10
5/5 [==============================] - 44s 9s/step - loss: 1.7523 - accuracy: 0.8836 - val_loss: 0.0889 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 42s 9s/step - loss: 0.0330 - accuracy: 1.0000 - val_loss: 0.0183 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 45s 10s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 47s 10s/step - loss: 1.7576e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 45s 10s/step - loss: 1.6824e-05 - accuracy: 1.0000 - val_loss: 5.0248e-04 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 43s 9s/step - loss: 6.5156e-06 - accuracy: 1.0000 - val_loss: 2.7766e-04 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 42s 9s/step - loss: 2.5627e-06 - accuracy: 1.0000 - val_loss: 1.9052e-04 - val_accuracy: 1.0000
Epo

In [13]:
from sklearn.metrics import confusion_matrix, classification_report


# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Define your class labels
class_labels = ['q_aff_cerris', 'q_rubur_f_purpubascens']

# Confusion Matrix
conf_mat = confusion_matrix(y_true_classes, y_pred_classes)


# Classification Report
class_report = classification_report(y_true_classes, y_pred_classes)

# Calculate Metrics
accuracy = (conf_mat[0, 0] + conf_mat[1, 1]) / np.sum(conf_mat)
precision = conf_mat[1, 1] / (conf_mat[1, 1] + conf_mat[0, 1])
recall = conf_mat[1, 1] / (conf_mat[1, 1] + conf_mat[1, 0])
f1_score = 2 * (precision * recall) / (precision + recall)
print(conf_mat)
print(class_report)
print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('F_ measure: ', f1_score)





2/2 [==============================] - 9s 3s/step
[[25  0]
 [ 0 24]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        24

    accuracy                           1.00        49
   macro avg       1.00      1.00      1.00        49
weighted avg       1.00      1.00      1.00        49

Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F_ measure:  1.0
